In [2]:
import pandas as pd

In [3]:
games = pd.read_csv("./clean_datasets/games.csv")  # gameid, title, genres, description
purchased_games = pd.read_csv("./clean_datasets/purchased_games.csv")  # Dữ liệu game đã mua
purchased_games['library'] = purchased_games['library'].apply(eval)


In [4]:
exploded = purchased_games.explode('library').rename(columns={'library': 'gameid'})  # explode library column to separate rows

purchases_per_game = exploded.groupby('gameid').size().reset_index(name='purchases_count')

# Gộp với bảng games
games_with_popularity = pd.merge(games, purchases_per_game, on='gameid', how='left').fillna(0)


In [5]:
games_with_popularity.to_csv("./clean_datasets/games_with_purchase_count.csv", index=False)

In [21]:
top_10_percent_games = games_with_popularity.sort_values(by="purchases_count", ascending=False).head(int(len(games) * 0.1))

In [ ]:
import os
if not os.path.exists("./data_model"):
    os.makedirs("./data_model")
    
top_10_percent_games.to_csv("./clean_datasets/top_10_percent_games.csv", index=False)